In [100]:
import pandas as pd
import requests

In [101]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1WasrN7Td_yzYTn9PLHe7PZHLsljDkkzJsOfhN5SPdZA'
SAMPLE_RANGE_NAME = 'VELKOBC2!A2:C73'


def main():
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'CREDENTIALS.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('sheets', 'v4', credentials=creds)

        # Call the Sheets API
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range=SAMPLE_RANGE_NAME).execute()
        values = result.get('values', [])

        if not values:
            print('No data found.')
            return
    
        return values
    except HttpError as err:
        print(err)


if __name__ == '__main__':
    values = main()

In [102]:
df = pd.DataFrame(values, columns =['Region', 'Population', 'No_of_Martinus'])

In [103]:
df.head()

,Region,Population,No_of_Martinus
0,Bratislava,435 295,3
1,District of Malacky,"74 323,0",0
2,District of Pezinok,"65 145,0",0
3,District of Senec,"89 832,0",0
4,District of Dunajská Streda,"122 358,0",0


In [104]:
df.dtypes

Region            object
Population        object
No_of_Martinus    object
dtype: object

In [105]:
df['No_of_Martinus'] = pd.to_numeric(df['No_of_Martinus'])

In [106]:
j = 0
for i in df['Population']:
    if type(i) is str:
        i = i.split(" ")
        i = "".join(i)
        i = i.replace(',', '.')
        df['Population'][j] = float(i)
    j += 1

<ipython-input-106-b45e2f0ef4dc>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Population'][j] = float(i)


In [107]:
df['Population'] = pd.to_numeric(df['Population'])

In [108]:
df.sort_values(by=['Population'], ascending=False)

,Region,Population,No_of_Martinus
0,Bratislava,435295.0,3
67,District of Košice,368218.5,2
61,District of Prešov,175609.5,1
22,District of Nitra,161560.0,1
28,District of Žilina,158043.0,1
...,...,...,...
48,District of Poltár,21471.0,0
52,District of Stropkov,20532.0,0
43,District of Banská Štiavnica,16086.0,0
37,District of Turčianske Teplice,15884.0,0


In [125]:
df_no_martinus = df[df["No_of_Martinus"]==0]

In [126]:
df_no_martinus = df_no_martinus.sort_values(by=['Population'], ascending=False)
df_no_martinus = df_no_martinus.reset_index(drop=True)
df_no_martinus.head()

,Region,Population,No_of_Martinus
0,District of Nové Zámky,139004.5,0
1,District of Prievidza,133979.5,0
2,District of Dunajská Streda,122358.0,0
3,District of Trenčín,114523.0,0
4,District of Levice,110824.0,0


KONIEC PARSOVANIA DAT

In [146]:
import requests
import json

knihkupectva = []
pomery = []
j=0
for mesto in df_no_martinus['Region']:
    mesto = mesto.split()[2:]
    mesto = "".join(mesto)
    url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?input={mesto}%20knihkupectvo&inputtype=textquery&radius=50000&type=bookstore&fields=name&key={MY_API_KEY}"
    # radius: 70km
    
    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    # print(mesto)
    knihkupectvo = [i['name'] for i in json.loads(response.text)['results']]
    # print(*knihkupectvo, sep=', ')
    # print("\n")
    knihkupectva.append(len(knihkupectvo))
    pomer = df_no_martinus['Population'][j]/len(knihkupectvo)
    pomery.append(pomer)
    j += 1



In [147]:
df_no_martinus['ine_knihkupectva'] =  knihkupectva
df_no_martinus['pomer'] = pomery
df_no_martinus.head()

,Region,Population,No_of_Martinus,ine_knihkupectva,pomer
0,District of Nové Zámky,139004.5,0,6,23167.416667
1,District of Prievidza,133979.5,0,14,9569.964286
2,District of Dunajská Streda,122358.0,0,5,24471.600000
3,District of Trenčín,114523.0,0,10,11452.300000
4,District of Levice,110824.0,0,6,18470.666667


In [148]:
df_no_martinus.sort_values(by=['pomer'], ascending=False)

,Region,Population,No_of_Martinus,ine_knihkupectva,pomer
13,District of Topoľčany,70131.5,0,1,70131.500000
27,District of Veľký Krtíš,43473.0,0,1,43473.000000
33,District of Kysucké Nové Mesto,32914.0,0,1,32914.000000
34,District of Detva,32051.0,0,1,32051.000000
36,District of Bytča,30917.0,0,1,30917.000000
10,District of Vranov nad Topľou,80766.5,0,3,26922.166667
37,District of Žarnovica,26152.5,0,1,26152.500000
22,District of Śaľa,51685.0,0,2,25842.500000
2,District of Dunajská Streda,122358.0,0,5,24471.600000
0,District of Nové Zámky,139004.5,0,6,23167.416667
